In [ ]:
from grid.robot.wheeled.airgen_car import AirGenCar
# initialise the robot
import airgen
from airgen.utils.collect import data_collector
from airgen import WeatherParameter, Vector3r
from typing import List, Tuple
airgen_car_0 = AirGenCar()

In [ ]:
# set the weather, wind, and timeofday parameters and import AI models
from grid.model.perception.detection.gdino import GroundingDINO
from grid.model.perception.segmentation.gsam2 import GSAM2

detection = GroundingDINO()
segmentation = GSAM2()

client = airgen_car_0.client

client.simEnableWeather(True)
client.simSetWeatherParameter(WeatherParameter.Fog, 1.0) # adds fog to the scene
client.simSetTimeOfDay(True, "2024-07-22 17:00:00") # sets the time of day to be around sunset

In [ ]:
search_radius = 50 # distance in meters
path = client.simPlanPathToRandomFreePoint(search_radius, smooth_path=True, draw_path=True) # generates the trajectory of points

points = []
for point in path:
    points.append(airgen.Vector3r(point['x_val'], point['y_val'], point['z_val']))

In [ ]:
def runAIModels(client: airgen.VehicleClient) -> dict:
    img, _ = client.getImages("front_center",[airgen.ImageType.Scene])[0]
    boxes, scores, labels = detection.run(img,"car.")
    mask = segmentation.run(img,"road")
    return img, boxes, scores, labels, mask

In [ ]:
@data_collector(runAIModels, time_delta=0.1)
def move_task(
    client: airgen.MultirotorClient, position: Tuple[float], **kwargs
) -> None | Tuple[None, List[dict]]:
    client.moveOnPath(points, velocity=5.0)

img, boxes, scores, labels, mask = move_task(client, (0, 0, -10), _collect_data=True)